In [224]:
# import libraries
import pandas as pd
import re

In [225]:
# load csv file to variable df as DataFrame
df = pd.read_csv('/Users/admin/database/data.csv')

In [226]:
# check how many rows and columns we have in file
df.shape

(187220, 10)

In [227]:
# display basic information (type, nullability, count) about our file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187220 entries, 0 to 187219
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   187220 non-null  int64  
 1   COUNTRY      187220 non-null  object 
 2   LINK         187220 non-null  object 
 3   COMPETITION  187220 non-null  object 
 4   TEAMS        187220 non-null  object 
 5   RESULT       187220 non-null  object 
 6   HOME         130306 non-null  float64
 7   DRAW         130306 non-null  float64
 8   AWAY         130306 non-null  float64
 9   DATE         187220 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 14.3+ MB


In [228]:
# show first 3 records
df.head(3)

,Unnamed: 0,COUNTRY,LINK,COMPETITION,TEAMS,RESULT,HOME,DRAW,AWAY,DATE
0,0,Germany,/handball/germany/bundesliga/,bundesliga,Bergischer - Erlangen,29:30,1.50,8.58,3.28,11.06.2023
1,1,Germany,/handball/germany/bundesliga/,bundesliga,Flensburg-H. - Rhein-Neckar,34:31,1.45,9.37,3.47,11.06.2023
2,2,Germany,/handball/germany/bundesliga/,bundesliga,Goppingen - Kiel,27:34,5.54,12.16,1.22,11.06.2023


In [229]:
# drop 2 columns which aren't necessary 
# - Unnamed:0 is ID from our database - now is our index automatic generated 
# - LINK has the same information which we can found COUNTRY and COMPETITION
df.drop(['Unnamed: 0','LINK'], axis=1, inplace=True)

In [230]:
df.head()

,COUNTRY,COMPETITION,TEAMS,RESULT,HOME,DRAW,AWAY,DATE
0,Germany,bundesliga,Bergischer - Erlangen,29:30,1.50,8.58,3.28,11.06.2023
1,Germany,bundesliga,Flensburg-H. - Rhein-Neckar,34:31,1.45,9.37,3.47,11.06.2023
2,Germany,bundesliga,Goppingen - Kiel,27:34,5.54,12.16,1.22,11.06.2023
3,Germany,bundesliga,Hamm-Westfalen - Leipzig,28:33,4.64,10.57,1.29,11.06.2023
4,Germany,bundesliga,HSG Wetzlar - SC Magdeburg,30:35,6.38,12.85,1.18,11.06.2023


In [232]:
# find value in column DATE with different mask than DD.MM.YYYY and drop it by index
# this step is obligatory because sometime when we scrape we didn't found 1 of 3 ODDS then ODD which we found was put in column DATE
i=0
for index, row in df.iterrows():
    if not bool(re.match(r'^\d{2}\.\d{2}\.\d{4}$', row['DATE'])):
        df.drop(index=index, inplace=True)
        i+=1
    else:
        pass
    
print(f'{i} records was dropped')

8 records was dropped


In [233]:
# now we can convert DATE from object to datetime
df['DATE'] = pd.to_datetime(df['DATE'], format='%d.%m.%Y', infer_datetime_format=True)

In [235]:
# we can see that 8 records mentioned above was dropped (invalid format of DATE) and type of DATE was changed
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187212 entries, 0 to 187219
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   COUNTRY      187212 non-null  object        
 1   COMPETITION  187212 non-null  object        
 2   TEAMS        187212 non-null  object        
 3   RESULT       187212 non-null  object        
 4   HOME         130306 non-null  float64       
 5   DRAW         130306 non-null  float64       
 6   AWAY         130306 non-null  float64       
 7   DATE         187212 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 12.9+ MB


In [244]:
# below we want to find some incorrect value in RESULT column
# RESULT of fixture should look like for exmaple 22:21 - so char ':' is necessary
# so find RESULT without this char and count values from this COLUMN
df.loc[~df['RESULT'].str.contains(':'),'RESULT'].count()

4203

In [246]:
# we see that we have 4203 value with invalid result
# CAN. - CANCEL - 4084
# WO. - WALKOVER - 93
# AWA. - AWARDED - 21
# POSTP - POSTPONED - 3
df.loc[~df['RESULT'].str.contains(':'),'RESULT'].value_counts()

CAN.      4084
WO.         93
AWA.        21
POSTP.       5
Name: RESULT, dtype: int64

In [247]:
# it isn't necessary to further analyze this records so we can remove it from our DataFrame
df=df[~df['RESULT'].isin(['CAN.','WO.','AWA.','POSTP.'])]

In [248]:
# we want to spli our result to 2 columns - column with home_team score and away_team score
# we should check that char : appear only once in this column - because we want to split it by this char
for index, row in df.iterrows():
    if len(row['RESULT'].split(':')) != 2:
        print(index)

In [256]:
# create 2 new columns to split RESULT by :
df[['H_score', 'A_score']] = df['RESULT'].str.split(':',expand=True)

In [257]:
# check actual status od DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183009 entries, 0 to 187219
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   COUNTRY      183009 non-null  object        
 1   COMPETITION  183009 non-null  object        
 2   TEAMS        183009 non-null  object        
 3   RESULT       183009 non-null  object        
 4   HOME         130152 non-null  float64       
 5   DRAW         130152 non-null  float64       
 6   AWAY         130152 non-null  float64       
 7   DATE         183009 non-null  datetime64[ns]
 8   H_score      183009 non-null  object        
 9   A_score      183009 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 15.4+ MB


In [258]:
# we want to convert form object to int score of H_team
df['H_score']=pd.to_numeric(df['H_score'])

In [259]:
# on right side of the RESULT sometimes we can notice some extra information so we can split it by space
# where extra information about RESULT didn't appear we should fill this by NULL
df[['A_score', 'INFO']] = df['A_score'].str.split(' ', expand=True).fillna('')

In [260]:
# the same instructions as 2 step before - convert result of A_team to int
df['A_score']=pd.to_numeric(df['A_score'])

In [263]:
# we can check - how much fixtures has some added information 
df['INFO'].value_counts()
# PEN - penatly
# ET - extra time 
# CAN - cancel 
# ABN /AWA - abandoned / awarded 
# WO - walkover 

        181133
PEN.       674
ET         639
AWA.       533
CAN.        22
ABN.         7
WO.          1
Name: INFO, dtype: int64

In [264]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183009 entries, 0 to 187219
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   COUNTRY      183009 non-null  object        
 1   COMPETITION  183009 non-null  object        
 2   TEAMS        183009 non-null  object        
 3   RESULT       183009 non-null  object        
 4   HOME         130152 non-null  float64       
 5   DRAW         130152 non-null  float64       
 6   AWAY         130152 non-null  float64       
 7   DATE         183009 non-null  datetime64[ns]
 8   H_score      183009 non-null  int64         
 9   A_score      183009 non-null  int64         
 10  INFO         183009 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 16.8+ MB


In [265]:
# column RESULT not needed anymore - we split information from this column to another (H_score, A_score and INFO)
df.drop(['RESULT'], axis=1, inplace=True)

In [266]:
df.head()

,COUNTRY,COMPETITION,TEAMS,HOME,DRAW,AWAY,DATE,H_score,A_score,INFO
0,Germany,bundesliga,Bergischer - Erlangen,1.50,8.58,3.28,2023-06-11,29,30,
1,Germany,bundesliga,Flensburg-H. - Rhein-Neckar,1.45,9.37,3.47,2023-06-11,34,31,
2,Germany,bundesliga,Goppingen - Kiel,5.54,12.16,1.22,2023-06-11,27,34,
3,Germany,bundesliga,Hamm-Westfalen - Leipzig,4.64,10.57,1.29,2023-06-11,28,33,
4,Germany,bundesliga,HSG Wetzlar - SC Magdeburg,6.38,12.85,1.18,2023-06-11,30,35,


In [270]:
# column known as TEAMS has 2 different team - we want to split it by ' - '
# check that we can split by this expression and get 2 columns
# if this expession not exist exactly 2 times we see value from this row
for index, row in df.iterrows():
    if len(row['TEAMS'].split(' - ')) != 2:
        print(row['TEAMS'])
    else:
        pass 

In [268]:
# we can see that only 4 rows are wrong - "TMBW - Berlin" is problematic so we remove extra spaces in name of this team
df.loc[df['TEAMS'].str.contains('TMBW - Berlin'),'TEAMS'] = df['TEAMS'].str.replace('TMBW - Berlin','TMBW-Berlin')

In [269]:
df.head()

,COUNTRY,COMPETITION,TEAMS,HOME,DRAW,AWAY,DATE,H_score,A_score,INFO
0,Germany,bundesliga,Bergischer - Erlangen,1.50,8.58,3.28,2023-06-11,29,30,
1,Germany,bundesliga,Flensburg-H. - Rhein-Neckar,1.45,9.37,3.47,2023-06-11,34,31,
2,Germany,bundesliga,Goppingen - Kiel,5.54,12.16,1.22,2023-06-11,27,34,
3,Germany,bundesliga,Hamm-Westfalen - Leipzig,4.64,10.57,1.29,2023-06-11,28,33,
4,Germany,bundesliga,HSG Wetzlar - SC Magdeburg,6.38,12.85,1.18,2023-06-11,30,35,


In [271]:
# we get 2 extra columns by split TEAMS by expression: ' - '
df[['H_team', 'A_team']] = df['TEAMS'].str.split(' - ',expand=True)

In [273]:
# column TEAMS not needed anymore - we split information from this column to another (H_team, A_team)
df.drop(['TEAMS'], axis=1, inplace=True)

In [276]:
df.head()

,COUNTRY,COMPETITION,HOME,DRAW,AWAY,DATE,H_score,A_score,INFO,H_team,A_team
0,Germany,bundesliga,1.50,8.58,3.28,2023-06-11,29,30,,Bergischer,Erlangen
1,Germany,bundesliga,1.45,9.37,3.47,2023-06-11,34,31,,Flensburg-H.,Rhein-Neckar
2,Germany,bundesliga,5.54,12.16,1.22,2023-06-11,27,34,,Goppingen,Kiel
3,Germany,bundesliga,4.64,10.57,1.29,2023-06-11,28,33,,Hamm-Westfalen,Leipzig
4,Germany,bundesliga,6.38,12.85,1.18,2023-06-11,30,35,,HSG Wetzlar,SC Magdeburg


In [277]:
# Country was from dictionary on database so it's correct
# we should check the last thing - competition which came from hyperlink to matches

In [278]:
# we have 1590 different Competition in our dataset
len(list(df['COMPETITION'].unique()))

1590

In [338]:
# we check exmaples of value from column known as COMPETITION
df['COMPETITION'].unique()

array(['Bundesliga', '2 Bundesliga', 'Dhb Pokal', 'Dhb Pokal Women',
       'Super Cup', 'Super Cup Women', '1 Bundesliga Women',
       '2 Bundesliga Women', 'Golden League Norway',
       'Golden League Netherlands Women', 'Ehf Euro Cup', 'European Cup',
       'European Cup Women', 'European Championship',
       'European Championship Women', 'Supercopa Iberica',
       'Golden League Denmark', 'Golden League Denmark Women',
       'Ehf Euro Cup Women', 'Bene League', 'Ehf Cup', 'Challenge Cup',
       'Challenge Cup Women', 'Mol Liga Women',
       'Golden League France Women', 'Golden League France',
       'Czech Slovak Cup', 'Golden League Norway Women', 'Ehf Cup Women',
       'Wrhl Women', 'Cup Winners Cup Women', 'Cup Winners Cup',
       '4 Nations Cup Denmark', 'Handbollsligan', 'Allsvenskan',
       'She Women', 'Allsvenskan Women', 'Elitserien', 'Santander Cup',
       'Santander Cup Women', '1 Division Women', 'Herre Handbold Ligaen',
       '1 Division', 'Hth Ligaen Wo

In [334]:
# we define pattern to extract SEASON which is on the end COMPETITION
pattern = r'\b(\d{4}(?:-\d{4})?)\b'
# Tworzenie nowej zmiennej
df['SEASON'] = df['COMPETITION'].str.extract(pattern)

In [335]:
# delete values which are now in column SEASON
for index, row in df.iterrows():
    df.at[index, 'COMPETITION'] = row['COMPETITION'].replace(str(row['SEASON']), '')

In [336]:
# Replace '-' by space, title and strip values
df['COMPETITION'] = df['COMPETITION'].str.replace('-',' ').str.title().str.strip()

In [339]:
df

,COUNTRY,COMPETITION,HOME,DRAW,AWAY,DATE,H_score,A_score,INFO,H_team,A_team,SEASON
0,Germany,Bundesliga,1.50,8.58,3.28,2023-06-11,29,30,,Bergischer,Erlangen,NaN
1,Germany,Bundesliga,1.45,9.37,3.47,2023-06-11,34,31,,Flensburg-H.,Rhein-Neckar,NaN
2,Germany,Bundesliga,5.54,12.16,1.22,2023-06-11,27,34,,Goppingen,Kiel,NaN
3,Germany,Bundesliga,4.64,10.57,1.29,2023-06-11,28,33,,Hamm-Westfalen,Leipzig,NaN
4,Germany,Bundesliga,6.38,12.85,1.18,2023-06-11,30,35,,HSG Wetzlar,SC Magdeburg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
187215,Ukraine,Superleague,4.20,11.00,1.28,2013-05-25,24,37,,Dinamo Poltava,Portovik,2012-2013
187216,Ukraine,Superleague,NaN,NaN,NaN,2013-05-19,32,29,,Motor Zaporozhye,Portovik,2012-2013
187217,Ukraine,Superleague,NaN,NaN,NaN,2013-05-19,27,17,,ZTR Zaporozhye,Dinamo Poltava,2012-2013
187218,Ukraine,Superleague,NaN,NaN,NaN,2013-05-14,24,30,,Dinamo Poltava,ZTR Zaporozhye,2012-2013


In [340]:
# define new order in DataFrame
new_order = ['COUNTRY', 'COMPETITION', 'SEASON','H_team','A_team','H_score','A_score','RESULT','INFO','HOME','DRAW','AWAY','DATE']
df = df.reindex(columns=new_order)

In [355]:
# matches from 2023 should be markes as SEASON 2022-2023 / this info wasn't in hyperlink
df.loc[df['DATE'].dt.year == 2023,'SEASON'] = '2022-2023'

In [365]:
# we still get some NaN value in SEASON because this was a first part of season 2022-2023
df.loc[df['SEASON'].isna()]

,COUNTRY,COMPETITION,SEASON,H_team,A_team,H_score,A_score,INFO,HOME,DRAW,AWAY,DATE
146,Germany,Bundesliga,NaN,Erlangen,Stuttgart,31,28,,1.41,9.50,3.73,2022-12-27
147,Germany,Bundesliga,NaN,Flensburg-H.,HSG Wetzlar,34,24,,1.08,17.19,10.25,2022-12-27
148,Germany,Bundesliga,NaN,Fuchse Berlin,Leipzig,28,22,,1.20,13.02,6.00,2022-12-27
149,Germany,Bundesliga,NaN,Gummersbach,Hamburg,31,30,,1.75,8.17,2.57,2022-12-27
150,Germany,Bundesliga,NaN,Hannover-Burgdorf,Bergischer,30,32,,1.40,9.28,3.86,2022-12-27
...,...,...,...,...,...,...,...,...,...,...,...,...
186597,Ukraine,Superleague,NaN,SKA Lviv,Donbas Mariupol,22,40,,NaN,NaN,NaN,2022-11-11
186598,Ukraine,Superleague,NaN,CSKA Kiev,Karpaty Uzhgorod,30,25,,NaN,NaN,NaN,2022-10-23
186599,Ukraine,Superleague,NaN,Odessa,Donbas Mariupol,24,33,,NaN,NaN,NaN,2022-10-22
186600,Ukraine,Superleague,NaN,SKA Lviv,CSKA Kiev,26,34,,NaN,NaN,NaN,2022-10-22


In [372]:
# we put value '2022-2023' in SEASON where is still NaN and DATE was from year 2022
df.loc[((df['DATE'].dt.year == 2022) & (df['SEASON'].isna())),'SEASON'] = '2022-2023'

In [374]:
# every season was filled
df['SEASON'].unique()

array(['2022-2023', '2021-2022', '2021', '2020-2021', '2020', '2019-2020',
       '2019', '2018-2019', '2018', '2017-2018', '2017', '2016-2017',
       '2016', '2015-2016', '2015', '2014-2015', '2014', '2013-2014',
       '2013', '2012-2013', '2012', '2011-2012', '2011', '2010-2011',
       '2009-2010', '2008-2009', '2007-2008', '2006-2007', '2005-2006',
       '2004-2005', '2003-2004', '2002-2003', '2001-2002', '2000-2001',
       '2022', nan, '2009', '2010', '2008', '1000', '2006', '2004',
       '2002', '2000', '1998', '2007', '2005', '2003', '2001', '1999',
       '1999-2000', '1998-1999'], dtype=object)

In [381]:
# create RESULT - dependencies on score
df.loc[df['H_score']>df['A_score'],'RESULT'] = 'H'

In [382]:
df.loc[df['H_score']<df['A_score'],'RESULT'] = 'A'

In [383]:
df.loc[df['H_score']==df['A_score'],'RESULT'] = 'D'

In [386]:
df.head(15)

,COUNTRY,COMPETITION,SEASON,H_team,A_team,H_score,A_score,INFO,HOME,DRAW,AWAY,DATE,RESULT
0,Germany,Bundesliga,2022-2023,Bergischer,Erlangen,29,30,,1.50,8.58,3.28,2023-06-11,A
1,Germany,Bundesliga,2022-2023,Flensburg-H.,Rhein-Neckar,34,31,,1.45,9.37,3.47,2023-06-11,H
2,Germany,Bundesliga,2022-2023,Goppingen,Kiel,27,34,,5.54,12.16,1.22,2023-06-11,A
3,Germany,Bundesliga,2022-2023,Hamm-Westfalen,Leipzig,28,33,,4.64,10.57,1.29,2023-06-11,A
4,Germany,Bundesliga,2022-2023,HSG Wetzlar,SC Magdeburg,30,35,,6.38,12.85,1.18,2023-06-11,A
5,Germany,Bundesliga,2022-2023,Hamburg,MT Melsungen,33,28,,1.67,7.93,2.78,2023-06-11,H
6,Germany,Bundesliga,2022-2023,Lemgo,Fuchse Berlin,35,32,,3.37,9.04,1.47,2023-06-11,H
7,Germany,Bundesliga,2022-2023,Minden,Gummersbach,38,38,,3.18,8.68,1.52,2023-06-11,D
8,Germany,Bundesliga,2022-2023,Stuttgart,Hannover-Burgdorf,31,34,,3.19,8.68,1.52,2023-06-11,A
9,Germany,Bundesliga,2022-2023,Erlangen,Flensburg-H.,28,33,,5.41,11.03,1.24,2023-06-08,A


In [387]:
# define new order in DataFrame
new_order = ['COUNTRY', 'COMPETITION', 'SEASON','H_team','A_team','H_score','A_score','RESULT','INFO','HOME','DRAW','AWAY','DATE']
df = df.reindex(columns=new_order)

In [391]:
df.head()

,COUNTRY,COMPETITION,SEASON,H_team,A_team,H_score,A_score,RESULT,INFO,HOME,DRAW,AWAY,DATE
0,Germany,Bundesliga,2022-2023,Bergischer,Erlangen,29,30,A,,1.50,8.58,3.28,2023-06-11
1,Germany,Bundesliga,2022-2023,Flensburg-H.,Rhein-Neckar,34,31,H,,1.45,9.37,3.47,2023-06-11
2,Germany,Bundesliga,2022-2023,Goppingen,Kiel,27,34,A,,5.54,12.16,1.22,2023-06-11
3,Germany,Bundesliga,2022-2023,Hamm-Westfalen,Leipzig,28,33,A,,4.64,10.57,1.29,2023-06-11
4,Germany,Bundesliga,2022-2023,HSG Wetzlar,SC Magdeburg,30,35,A,,6.38,12.85,1.18,2023-06-11


In [392]:
# save the file
df.to_csv('handball.csv', index=False)